# `CLOUD.jl` - 2D linear advection example

In [8]:
using CLOUD, OrdinaryDiffEq

## Define the physical problem

In [9]:
θ = π/4 # wave angle
a = sqrt(2) # wave speed
A = 1.0  # amplitude
L = 1.0  # domain length
k = (2*π/L, 2*π/L)  # wave number
T = 1.0/(a*max(abs(cos(θ)),abs(sin(θ)))) # end time for one period

conservation_law = LinearAdvectionEquation((a*cos(θ),a*sin(θ)))
initial_data = InitialDataSine(A,k)
exact_solution = ExactSolution(conservation_law,initial_data);

## Set discretization parameters

In [10]:
M = 4
p = 4
p_map = 4
form = WeakConservationForm(mapping_form=SkewSymmetricMapping(), 
    inviscid_numerical_flux=LaxFriedrichsNumericalFlux(0.0))
strategy = ReferenceOperator()

ode_algorithm = CarpenterKennedy2N54()
dt = 0.01*(L/M)/(a*(2*p+1));
write_interval = floor(Int, T/(dt*10));

## Set up a `DGSEM` scheme with LG quadrature on quadrilaterals of degree `p` with `M` edges in each direction

In [11]:
#reference_approximation = ReferenceApproximation(
#    DGSEM(p), Quad(), mapping_degree=p_map, N_plot=5)

reference_approximation = ReferenceApproximation(
    Multidimensional(p), Tri(), mapping_degree=p_map, N_plot=2)

mesh = warp_mesh(uniform_periodic_mesh(
    reference_approximation.reference_element, 
    ((0.0,L),(0.0,L)), (M,M)), 
    reference_approximation.reference_element, 0.2)

spatial_discretization = SpatialDiscretization(mesh, 
    reference_approximation)

results_path = save_project(conservation_law,
     spatial_discretization, initial_data, form, (0.0, T), strategy,
     "results/advection_2d_dgsem/", overwrite=true, clear=true)

"results/advection_2d_dgsem/"

## Run the solver

In [12]:
ode_problem_1 = semidiscretize(conservation_law,
    spatial_discretization,
    initial_data, 
    form,
    (0.0, T),
    strategy);

save_solution(ode_problem_1.u0, 0.0, results_path, 0)

CLOUD_reset_timer!()
sol = solve(ode_problem_1, ode_algorithm, adaptive=false, dt=dt,
    save_everystep=false, callback=save_callback(results_path, write_interval))
CLOUD_print_timer()

save_solution(last(sol.u), last(sol.t), results_path, "final")

 ────────────────────────────────────────────────────────────────────────────────
            Thread 1                    Time                    Allocations      
                               ───────────────────────   ────────────────────────
       Tot / % measured:            20.5s /  92.5%           17.8GiB /  97.2%    

 Section               ncalls     time    %tot     avg     alloc    %tot      avg
 ────────────────────────────────────────────────────────────────────────────────
 local residual          815k    17.5s   92.4%  21.5μs   16.6GiB   96.2%  21.4KiB
   apply operators       815k    9.06s   47.9%  11.1μs   9.03GiB   52.3%  11.6KiB
     volume terms        815k    3.39s   17.9%  4.16μs   4.93GiB   28.6%  6.34KiB
     mass matrix solve   815k    2.03s   10.7%  2.50μs   1.81GiB   10.5%  2.33KiB
     facet terms         815k    619ms    3.3%   760ns    870MiB    4.9%  1.09KiB
     mul test function   815k    383ms    2.0%   470ns    323MiB    1.8%     416B
   gather ext s

## Calculate the integral L<sup>2</sup> error norm and conservation/energy error

In [13]:
error_analysis = ErrorAnalysis(results_path, conservation_law, 
    spatial_discretization)
conservation_analysis = PrimaryConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)
energy_analysis = EnergyConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)

println("L2 error:")
println(analyze(error_analysis, last(sol.u), initial_data, T)...)
println("Conservation (initial/final/diff):")
println(analyze(conservation_analysis)...)
println("Energy (initial/final/diff):")
println(analyze(energy_analysis)...)

L2 error:
0.041944328206357184
Conservation (initial/final/diff):
[-1.8792023555769862e-5][-1.879202355690177e-5][-1.1319070680748666e-15]
Energy (initial/final/diff):
[0.12490255166391541][0.12490255166387607][-3.9343528435153985e-14]


## Write output to VTK file

In [14]:
postprocess_vtk(spatial_discretization, string(results_path,"solution.vtu"), last(sol.u));